In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data

In [2]:
from sqlalchemy import create_engine


driver = 'mysql+pymysql:'
user = 'adria'
password = '00000'
ip = '35.187.114.125'
database = 'vimet'

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [3]:
items = pd.read_sql('items', con = engine)

In [4]:
items = items.drop(columns='index')

In [6]:
items.head()

,Name,Lineitem quantity,Lineitem name,Lineitem price,Lineitem fulfillment status,Cancelled at,Refunded Amount,Vendor,Lineitem discount,tags
0,#1248,4,Plátano Canarias - 3/4 unidades,1.95,fulfilled,not cancelled,0.0,Fruites i Verdures Rovira,0,Plátano
1,#1248,2,Fresón Maresme - 1 caja (500 grs.),3.50,fulfilled,not cancelled,0.0,Fruites i Verdures Rovira,0,Fresón
2,#1248,1,Tomate Cherry - 200 grs,1.30,fulfilled,not cancelled,0.0,Fruites i Verdures Rovira,0,Tomate
3,#1248,1,Nueces California sin cáscara - 200 grs,3.44,fulfilled,not cancelled,0.0,Llegums Porta Novau,0,Nuez
4,#1248,3,Mandarina Orri - 500 grs.,2.00,fulfilled,not cancelled,0.0,Fruites i Verdures Rovira,0,Mandarina


In [8]:
len(items['tags'].value_counts())

154

We have 154 different items, we will cluster them creating a tag2 with more general terms, according to a reference on mediterranean diet:
* Dulces
* Patata
* Carne roja
* Carne procesada
* Carne blanca
* Pescado
* Huevos
* Legumbres
* Lácteos
* Especias
* Frutos secos
* Frutas
* Verduras
* Cereales
* Aceite

In [13]:
# We create a dicitonary classifying all the items
dict_tags = {'Dulces':['Obleas', 'Chocolate'], 
             'Patata': ['Patata'],
             'Carne roja': ['Ternera', 'Confit', 'Magret', 'Vaca', 'Filet', 'Solomillo', 'Entrecot'], 
             'Carne procesada': ['Jamón', 'Croquetas', 'Bull', 'Lasagna', 'Chorizo', 'Sumaia', 'salchichón',
                                 'Mortadela', 'Albóndigas', 'Librito', 'Butifarra', 'Salchichas', 'Canelones',
                                'Fuet', 'Espectec', 'Sobrasada', 'Bull'], 
             'Carne blanca': ['Pollo', 'Pavo', 'Conejo', 'Cerdo', 'Codorniz', 'Costillas'], 
             'Pescado': ['Mero', 'Bonito', 'Merluza', 'Salmón', 'Dorada', 'Mejillón', 'Lubina', 'Calamar', 'Sepia',
                        'Caballa', 'Lenguado', 'Almejas', 'Rape', 'Salmonetes', 'Boquerones', 'Berberechos', 'Atun',
                        'Gamba', 'Sardina', 'Pescadilla'], 
             'Huevos': ['Huevos', 'Revuelto', 'Tortilla'], 
             'Legumbres':['Garbanzo', 'Lenteja', 'Judía', 'Montgeta', 'Alubia', 'Frijoles', 'Azuki', 'Hummus'], 
             'Lácteos': ['Queso', 'Yogures'], 
             'Especias':['Eneldo', 'Ajo', 'Picantón'], 
             'Frutos secos': ['Nuez', 'Almendra', 'Macadamia', 'Avellana', 'Cacahuetes', 'Pasa', 'Dátiles',
                             'Anacardo', 'Pistacho', 'Orejones'], 
             'Frutas': ['Plátano','Manzana', 'Fresón', 'Naranja', 'Pera', 'Kiwi', 'Paraguayo', 'Mango', 'Chirimoya',
                        'Kakis', 'Ciruela', 'Mandarina',  'Melocotón', 'Piña','Arándanos', 'Limón', 'Bananas',
                       'Albaricoque', 'Granada', 'Higos', 'Cerezas', 'Frambuesas', 'Sandía', 'Nectarina', 'Pomelo',
                        'Uva', 'Melón', 'Paraguayo'], 
             'Verduras':['Tomate', 'Cebolla', 'Calabacín','Zanahoria', 'Pimiento', 'Aguacate', 'Brócoli', 'Lechuga',
                         'Berenjena', 'Calçots', 'Escarola' 'Cogollos', 'Calabaza', 'Acelgas', 'Espinacas','Puerro',
                         'Endivias', 'Rúcula', 'Alcachofa', 'Esparrago', 'Pepino', 'Canónigos', 'Ensalada', 'Coliflor',
                        'Col', 'Apio'],
             'Cereales':['Granola', 'Soja', 'Mijo', 'Avena', 'Quinoa', 'Arroz', 'Pan', 'Macarrones', 'Pasta',
                        'Spaghetti', 'Cous-cous', 'Espaguetis'],
             'Aceite':['Aceite'],
             'Others': ['Cep', 'Agua', 'Salsa', 'Girgola', 'Champiñón', 'Caldo', 'Shiitake', 'Rossinyol']}

In [46]:
#We reverse (keys for values), the dictionary so that we can easily assign it to a new series.
dict_tags2 = {}

for key, values in dict_tags.items():
    for value in values:
        dict_tags2[value] = key

In [49]:
#We create a new serie with the 'group tags'
items['group_tags'] = items['tags'] 
items['group_tags'] = items['group_tags'].map(dict_tags2) 